In [1]:
# To display full output in Notebook, instead of only the last result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import requests
import json
import time
from bech32 import bech32_encode, convertbits

from datetime import datetime
from tqdm import tqdm
from bech32 import bech32_decode, convertbits
from binascii import hexlify

In [3]:
pd.options.display.float_format = "{:.6f}".format

In [4]:
def bech32_to_hex(addr_bech32: str) -> str:
    hrp, data = bech32_decode(addr_bech32)
    if data is None:
        raise ValueError(f"Invalid bech32 address: {addr_bech32}")
    decoded = convertbits(data, 5, 8, False)
    return hexlify(bytes(decoded)).decode()

def safe_bech32_to_hex(addr):
    if pd.isna(addr):
        return None
    try:
        return bech32_to_hex(addr)
    except Exception:
        return None

def hex_to_bech32(addr_hex: str, hrp="addr") -> str:
    """Convert 128-char hex string to bech32 address."""
    data = bytes.fromhex(addr_hex)
    five_bit = convertbits(data, 8, 5)
    return bech32_encode(hrp, five_bit)


def safe_hex_to_bech32(addr_hex: str, hrp="addr") -> str:
    """Convert 128-char hex string to bech32 address with error handling."""
    try:
        data = bytes.fromhex(addr_hex)
        five_bit = convertbits(data, 8, 5)
        return bech32_encode(hrp, five_bit)
    except Exception as e:
        return None  

In [5]:
path = "/home/jovyan/work/New Topic/MIN_holders_df.csv"
MIN_holders_df = pd.read_csv(path)

In [6]:
MIN_holders_df.head()

,token,bech32_address,quantity,token_amount,hex_address
0,MIN,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,998422624088847,998422624.088847,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...
1,MIN,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,428049739523097,428049739.523097,716563d77a4da81a91831c68d192a3eeefb0340f134151...
2,MIN,addr1z84q0denmyep98ph3tmzwsmw0j7zau9ljmsqx6a4r...,234793515755785,234793515.755785,11ea07b733d932129c378af627436e7cbc2ef0bf96e003...
3,MIN,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,229125022327324,229125022.327324,11fcc769708aae4772b2ad896281286c742b88d127938f...
4,MIN,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,207620627540886,207620627.540886,71229863ae8694da16da002e889a1d19bcc82c581225b6...


In [7]:
# Count how many bech32 addresses start with "addr1v"
enterprise_addr_count = MIN_holders_df["bech32_address"].str.startswith("addr1v").sum()
print(f"Number of Enterprise (addr1v...) addresses: {enterprise_addr_count}")

Number of Enterprise (addr1v...) addresses: 67


In [8]:
MIN_holders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25840 entries, 0 to 25839
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   token           25840 non-null  object 
 1   bech32_address  25840 non-null  object 
 2   quantity        25840 non-null  int64  
 3   token_amount    25840 non-null  float64
 4   hex_address     25804 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1009.5+ KB


# API Fetching Txs Summaries

In [9]:
def bech32_to_hex(bech32_addr):
    """
    Convert a bech32 Shelley address (e.g., addr1...) to hex.
    """
    hrp, data = bech32_decode(bech32_addr)
    if hrp is None or data is None:
        raise ValueError("Invalid bech32 address")
    decoded_bytes = bytes(convertbits(data, 5, 8, False))
    return decoded_bytes.hex()

def safe_bech32_to_hex(addr):
    try:
        return bech32_to_hex(addr)
    except Exception:
        return None

In [10]:
from collections import Counter

all_addresses = MIN_holders_df["bech32_address"].dropna().astype(str)

def classify_prefix(addr):
    if addr.startswith("DdzFF"):
        return "Byron (DdzFF)"
    elif addr.startswith("Ae2td"):
        return "Byron (Ae2td)"
    elif addr.startswith("addr1"):
        return "Shelley"
    elif addr.startswith("stake1"):
        return "Stake"
    else:
        return "Other / Unknown"

prefix_counts = Counter([classify_prefix(a) for a in all_addresses])
print(prefix_counts)


Counter({'Shelley': 25804, 'Byron (DdzFF)': 18, 'Byron (Ae2td)': 18})


In [11]:
import requests
import json
import time
from tqdm.notebook import tqdm
from bech32 import bech32_decode, convertbits  # pip install bech32


def fetch_blockfrost_address_summaries(addresses, output_file="min_address_txs_summary.json",
                                       api_key="mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu",
                                       max_retries=3, sleep_time=0.2):
    headers = {"project_id": api_key}
    
    # ✅ Filter only valid bech32 addresses (exclude Byron)
    valid_addresses = [addr for addr in addresses if safe_bech32_to_hex(addr) is not None]

    summary = {}
    bar = tqdm(total=len(valid_addresses), desc="Fetching summaries")

    for addr in valid_addresses:
        for _ in range(max_retries):
            try:
                res = requests.get(f"https://cardano-mainnet.blockfrost.io/api/v0/addresses/{addr}/total", headers=headers)
                if res.status_code == 200:
                    summary[addr] = res.json()
                    break
            except Exception:
                pass
            time.sleep(sleep_time)
        bar.update(1)

    bar.close()

    successful_data = {k: v for k, v in summary.items() if "error" not in v}

    with open(output_file, "w") as f:
        json.dump(successful_data, f, indent=2)

    print(f"✅ Done! {len(successful_data)} results saved to '{output_file}'")

# 🟡 Replace with your real list
address_list = MIN_holders_df["bech32_address"].dropna().unique().tolist()
fetch_blockfrost_address_summaries(address_list)


Fetching summaries:   0%|          | 0/25804 [00:00<?, ?it/s]

✅ Done! 25793 results saved to 'min_address_txs_summary.json'


In [12]:
valid_addresses = [addr for addr in address_list if safe_bech32_to_hex(addr) is not None]
dropped_addresses = set(address_list) - set(valid_addresses)
print(f"❌ Dropped {len(dropped_addresses)} invalid addresses")


❌ Dropped 36 invalid addresses


# Get Address Txs Summary

In [7]:
import json
import pandas as pd

# === Load JSON ===
with open("min_address_txs_summary.json") as f:
    data = json.load(f)

# === Token policy ID and decimals mapping ===
token_policies = {
    "GENS": "dda5fdb1002f7389b33e036b6afee82a8189becb6cba852e8b79b4fb0014df1047454e53",
    "MILK": "8a1cfae21368b8bebbbed9800fec304e95cce39a2a57dc35e2e3ebaa4d494c4b",
    "MIN": "29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e",
    "WRT": "c0ee29a85b13209423b10447d3c2e6a50641a15c57770e27cb9d507357696e67526964657273",
    "SUNDAE": "9a9693a9a37912a5097918f97918d15240c92ab729a0b7c4aa144d7753554e444145"
}

token_decimals = {
    "GENS": 6,
    "MILK": 0,
    "MIN": 6,
    "WRT": 6,
    "SUNDAE": 6
}

# === Extract records ===
# === Extract records ===
records = []

for addr, info in data.items():
    row = {"bech32_address": addr}

    # Transaction count
    row["tx_count"] = info.get("tx_count", 0)

    # Initialize cumulative values
    row["received_sum_ada"] = 0
    row["sent_sum_ada"] = 0
    for token in token_policies:
        row[f"received_sum_{token.lower()}"] = 0
        row[f"sent_sum_{token.lower()}"] = 0

    # Use sets to record unique token units (excluding lovelace)
    received_units = set()
    sent_units = set()

    # Parse received amounts
    for item in info.get("received_sum", []):
        unit = item["unit"]
        qty = int(item["quantity"])
        if unit == "lovelace":
            row["received_sum_ada"] += qty / 1_000_000
        else:
            received_units.add(unit)  # Track all unique tokens (no whitelist)
            for token, policy in token_policies.items():
                if unit.startswith(policy):
                    decimals = token_decimals[token]
                    row[f"received_sum_{token.lower()}"] += qty / (10 ** decimals)

    # Parse sent amounts
    for item in info.get("sent_sum", []):
        unit = item["unit"]
        qty = int(item["quantity"])
        if unit == "lovelace":
            row["sent_sum_ada"] += qty / 1_000_000
        else:
            sent_units.add(unit)  # Track all unique tokens (no whitelist)
            for token, policy in token_policies.items():
                if unit.startswith(policy):
                    decimals = token_decimals[token]
                    row[f"sent_sum_{token.lower()}"] += qty / (10 ** decimals)

    # Count unique tokens (excluding ADA/lovelace)
    row["received_token_count"] = len(received_units)
    row["sent_token_count"] = len(sent_units)

    records.append(row)


# === Convert to DataFrame
min_holders_txs_summary_df = pd.DataFrame(records)

# ✅ Done!
min_holders_txs_summary_df.head()


,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,received_sum_wrt,sent_sum_wrt,received_sum_sundae,sent_sum_sundae,received_token_count,sent_token_count
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619873,24556163659.531025,0.000000,0.000000,0.000000,0.000000,9,1
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,0.000000,0.000000,0.000000,0.000000,1,1
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,0.000000,0.000000,0.000000,0.000000,2,1
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,0.000000,0.000000,0.000000,0.000000,1,1
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,0.000000,0.000000,0.000000,0.000000,3,1


In [8]:
min_holders_txs_summary_df["hex_address"] = min_holders_txs_summary_df["bech32_address"].apply(safe_bech32_to_hex)

# ✅ Done!
min_holders_txs_summary_df.head()

,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,received_sum_wrt,sent_sum_wrt,received_sum_sundae,sent_sum_sundae,received_token_count,sent_token_count,hex_address
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619873,24556163659.531025,0.000000,0.000000,0.000000,0.000000,9,1,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,0.000000,0.000000,0.000000,0.000000,1,1,716563d77a4da81a91831c68d192a3eeefb0340f134151...
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,0.000000,0.000000,0.000000,0.000000,2,1,11fcc769708aae4772b2ad896281286c742b88d127938f...
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,0.000000,0.000000,0.000000,0.000000,1,1,71229863ae8694da16da002e889a1d19bcc82c581225b6...
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,0.000000,0.000000,0.000000,0.000000,3,1,115cd650b00784222bfa462e08aabef8bd082606856939...


In [9]:
min_holders_txs_summary_df['hex_address'][0]

'119ee377fa68bc3908d656dc9648b4d54612f3b200e184acd5aac3abf752563c5410bff6a0d43ccebb7c37e1f69f5eb260552521adff33b9c2'

In [10]:
min_holders_txs_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bech32_address        25793 non-null  object 
 1   tx_count              25793 non-null  int64  
 2   received_sum_ada      25793 non-null  float64
 3   sent_sum_ada          25793 non-null  float64
 4   received_sum_gens     25793 non-null  float64
 5   sent_sum_gens         25793 non-null  float64
 6   received_sum_milk     25793 non-null  float64
 7   sent_sum_milk         25793 non-null  float64
 8   received_sum_min      25793 non-null  float64
 9   sent_sum_min          25793 non-null  float64
 10  received_sum_wrt      25793 non-null  float64
 11  sent_sum_wrt          25793 non-null  float64
 12  received_sum_sundae   25793 non-null  float64
 13  sent_sum_sundae       25793 non-null  float64
 14  received_token_count  25793 non-null  int64  
 15  sent_token_count   

In [13]:
min_holders_txs_summary_df['hex_address'].nunique()

25793

# Get Address Assets Summary

In [14]:
# Your API key
CARDANOSCAN_API_KEY = "520c718b-75dc-4898-aeca-199e059de866"

In [17]:
import requests
import json
import time
from tqdm.notebook import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# === Setup session with retry and backoff ===
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1.0,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)

# === API ===
CARDANOSCAN_API_KEY = "e18f58b0-7c89-43c9-aa34-c479ac025c74"
HEADERS = {"apiKey": CARDANOSCAN_API_KEY}
BASE_URL = "https://api.cardanoscan.io/api/v1/asset/list/byAddress"

# === Addresses ===
hex_addresses = min_holders_txs_summary_df["hex_address"].dropna().unique().tolist()
address_assets = {}
failed_addresses = []  # use to record failed address

print(f"🔍 Fetching asset lists for {len(hex_addresses)} addresses...")

for addr in tqdm(hex_addresses, desc="Fetching assets by address"):
    all_assets = []
    page = 1
    success = True  # default to be successful

    while True:
        url = f"{BASE_URL}?address={addr}&pageNo={page}"
        try:
            response = session.get(url, headers=HEADERS, timeout=15)

            if response.status_code == 200:
                try:
                    data = response.json()
                    tokens = data.get("tokens", [])
                except json.JSONDecodeError:
                    print(f"⚠️ JSON decode error for {addr} page {page}")
                    success = False  # sign to be failed
                    break

                if not tokens:
                    break  #  No more tokens

                all_assets.extend(tokens)
                page += 1
                time.sleep(1.0)

            else:
                print(f"❌ Failed for {addr} | Status: {response.status_code} | Body: {response.text}")
                success = False  # sign to be failed
                break

        except requests.exceptions.SSLError as e:
            print(f"🔒 SSL error for {addr} page {page}: {e}")
            success = False
            break

        except requests.exceptions.RequestException as e:
            print(f"⚠️ Request exception for {addr}: {e}")
            success = False
            break

    address_assets[addr] = all_assets

    if not success:  # if it's failed, record it
        failed_addresses.append(addr)

# === Save results
with open("min_address_assets_by_hex.json", "w") as f:
    json.dump(address_assets, f, indent=2)

with open("failed_addresses.json", "w") as f:  # save failed address
    json.dump(failed_addresses, f, indent=2)

print("✅ Done!")
print(f"🧯 {len(failed_addresses)} addresses failed and were saved to 'failed_addresses.json'")


🔍 Fetching asset lists for 25793 addresses...


Fetching assets by address:   0%|          | 0/25793 [00:00<?, ?it/s]

✅ Done!
🧯 0 addresses failed and were saved to 'failed_addresses.json'


In [11]:
import json
import pandas as pd

# === Load JSON ===
with open("min_address_assets_by_hex.json", "r") as f:
    address_assets = json.load(f)

# === Asset ID mapping ===
asset_id_map = {
    "dda5fdb1002f7389b33e036b6afee82a8189becb6cba852e8b79b4fb0014df1047454e53": "GENS",
    "8a1cfae21368b8bebbbed9800fec304e95cce39a2a57dc35e2e3ebaa4d494c4b": "MILK",
    "29d222ce763455e3d7a09a665ce554f00ac89d2e99a1a83d267170c64d494e": "MIN",
    "c0ee29a85b13209423b10447d3c2e6a50641a15c57770e27cb9d507357696e67526964657273": "WRT",
    "9a9693a9a37912a5097918f97918d15240c92ab729a0b7c4aa144d7753554e444145": "SUNDAE"
}

# === Token Decimals ===
token_decimals = {
    "WRT": 6,
    "SUNDAE": 6,
    "MILK": 0,
    "MIN": 6,
    "GENS": 6,
}

# === Container for each row ===
records = []

# === Parse each address ===
for hex_address, tokens in address_assets.items():
    token_counts = len(tokens)
    balances = {
        "gens_balance": 0,
        "milk_balance": 0,
        "min_balance": 0,
        "wrt_balance": 0,
        "sundae_balance": 0,
    }

    for token in tokens:
        asset_id = token.get("assetId")
        balance_raw = int(token.get("balance", 0))
        label = asset_id_map.get(asset_id)

        if label:  # If this is one of the 5 tracked tokens
            decimals = token_decimals[label]
            readable_balance = balance_raw / (10 ** decimals)
            balances[f"{label.lower()}_balance"] = readable_balance

    record = {
        "hex_address": hex_address,
        **balances,
        "token_num": token_counts
    }
    records.append(record)

# === Convert to DataFrame ===
min_holders_assets_summary_df = pd.DataFrame(records)

# === Preview result ===
min_holders_assets_summary_df.head()


,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num
0,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...,0.000000,0.000000,998422624.088847,0.000000,0.000000,9
1,716563d77a4da81a91831c68d192a3eeefb0340f134151...,0.000000,0.000000,446878456.418966,0.000000,0.000000,1
2,11fcc769708aae4772b2ad896281286c742b88d127938f...,0.000000,0.000000,229125022.327324,0.000000,0.000000,2
3,71229863ae8694da16da002e889a1d19bcc82c581225b6...,0.000000,0.000000,196018403.962452,0.000000,0.000000,1
4,115cd650b00784222bfa462e08aabef8bd082606856939...,0.000000,0.000000,199077142.860000,0.000000,0.000000,3


In [12]:
min_holders_assets_summary_df.tail()

,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num
25788,015d4ece420a178ae281028b2e64e9f42af5a08b9d7859...,0.000000,0.000000,1.000000,0.000000,0.000000,11
25789,01c927c287693d5bfb72e451fb3e139fafba3ba7f4cc52...,0.000000,0.000000,1.000000,0.000000,0.000000,3
25790,0115bb87cd022b537602752557be19adb4d3a3982e2044...,0.000000,0.000000,1.000000,0.000000,0.000000,1
25791,01efd7da1bbdfd4ed2f80bb11270d2ade7212723b8ac5a...,0.000000,0.000000,1.000000,201.738319,780.738696,103
25792,018d5b5d6c4b4dec57c4743011b9458773cac0c41bc8b3...,0.000000,0.000000,1.000000,0.000000,0.000000,1


In [13]:
min_holders_assets_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   hex_address     25793 non-null  object 
 1   gens_balance    25793 non-null  float64
 2   milk_balance    25793 non-null  float64
 3   min_balance     25793 non-null  float64
 4   wrt_balance     25793 non-null  float64
 5   sundae_balance  25793 non-null  float64
 6   token_num       25793 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 1.4+ MB


In [14]:
# === Merge the two dataframes on 'hex_address' ===
min_holders_txs_and_assets_summary_df = pd.merge(
    min_holders_txs_summary_df,
    min_holders_assets_summary_df,
    on="hex_address",
    how="left" 
)

min_holders_txs_and_assets_summary_df.head()


,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,sent_sum_sundae,received_token_count,sent_token_count,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619873,24556163659.531025,...,0.000000,9,1,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...,0.000000,0.000000,998422624.088847,0.000000,0.000000,9
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,...,0.000000,1,1,716563d77a4da81a91831c68d192a3eeefb0340f134151...,0.000000,0.000000,446878456.418966,0.000000,0.000000,1
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,...,0.000000,2,1,11fcc769708aae4772b2ad896281286c742b88d127938f...,0.000000,0.000000,229125022.327324,0.000000,0.000000,2
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,...,0.000000,1,1,71229863ae8694da16da002e889a1d19bcc82c581225b6...,0.000000,0.000000,196018403.962452,0.000000,0.000000,1
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,...,0.000000,3,1,115cd650b00784222bfa462e08aabef8bd082606856939...,0.000000,0.000000,199077142.860000,0.000000,0.000000,3


In [15]:
min_holders_txs_and_assets_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bech32_address        25793 non-null  object 
 1   tx_count              25793 non-null  int64  
 2   received_sum_ada      25793 non-null  float64
 3   sent_sum_ada          25793 non-null  float64
 4   received_sum_gens     25793 non-null  float64
 5   sent_sum_gens         25793 non-null  float64
 6   received_sum_milk     25793 non-null  float64
 7   sent_sum_milk         25793 non-null  float64
 8   received_sum_min      25793 non-null  float64
 9   sent_sum_min          25793 non-null  float64
 10  received_sum_wrt      25793 non-null  float64
 11  sent_sum_wrt          25793 non-null  float64
 12  received_sum_sundae   25793 non-null  float64
 13  sent_sum_sundae       25793 non-null  float64
 14  received_token_count  25793 non-null  int64  
 15  sent_token_count   

In [16]:
min_holders_txs_and_assets_summary_df['hex_address'][0]

'119ee377fa68bc3908d656dc9648b4d54612f3b200e184acd5aac3abf752563c5410bff6a0d43ccebb7c37e1f69f5eb260552521adff33b9c2'

# Get ADA Balance

In [33]:
!curl \
 --request GET 'https://api.cardanoscan.io/api/v1/address/balance?address=11af97793b8702f381976cec83e303e9ce17781458c73c4bb16fe02b831ffbacc6c0d1ac76e1562d400a8e991eede2210c691fb433009171b0' \
 --header "apiKey: 520c718b-75dc-4898-aeca-199e059de866"

{"hash":"11af97793b8702f381976cec83e303e9ce17781458c73c4bb16fe02b831ffbacc6c0d1ac76e1562d400a8e991eede2210c691fb433009171b0","balance":"355231554617"}

In [24]:
import requests
import time
import json
from tqdm import tqdm

# === Setup ===
API_KEY = "520c718b-75dc-4898-aeca-199e059de866"  
HEADERS = {"apiKey": API_KEY}
BASE_URL = "https://api.cardanoscan.io/api/v1/address/balance"

# === Get all unique hex addresses ===
hex_addresses = min_holders_txs_and_assets_summary_df["hex_address"].dropna().unique().tolist()

# === Container for results ===
ada_balance_dict = {}

# === Fetch ADA balance for each address ===
print(f"🔍 Fetching ADA balance for {len(hex_addresses)} addresses...")

for addr in tqdm(hex_addresses):
    for attempt in range(3):  # retry up to 3 times
        try:
            response = requests.get(f"{BASE_URL}?address={addr}", headers=HEADERS)
            if response.status_code == 200:
                data = response.json()
                balance = int(data.get("balance", 0))
                ada_balance_dict[addr] = balance
                break
            else:
                print(f"⚠️ {addr} | Status: {response.status_code}")
                time.sleep(1)
        except Exception as e:
            print(f"❌ {addr} | Error: {e}")
            time.sleep(1)
    time.sleep(0.2)  # rate limiting

# === Save as JSON file ===
with open("min_address_ada_balance.json", "w") as f:
    json.dump(ada_balance_dict, f, indent=2)

print("✅ Saved ADA balances to min_address_ada_balance.json")


🔍 Fetching ADA balance for 25793 addresses...


100%|███████████████████████████████████| 25793/25793 [5:18:25<00:00,  1.35it/s]


✅ Saved ADA balances to min_address_ada_balance.json


In [17]:
import json

# Step 1: Load ADA balances from JSON
with open("min_address_ada_balance.json", "r") as f:
    ada_balance_dict = json.load(f)

# Step 2: Map and convert to ADA in one step
min_holders_txs_and_assets_summary_df["ada_balance"] = (
    min_holders_txs_and_assets_summary_df["hex_address"].map(ada_balance_dict) / 1_000_000
)

# Optional: Check result
min_holders_txs_and_assets_summary_df.head()


,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,received_token_count,sent_token_count,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num,ada_balance
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619873,24556163659.531025,...,9,1,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...,0.000000,0.000000,998422624.088847,0.000000,0.000000,9,5.766780
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,...,1,1,716563d77a4da81a91831c68d192a3eeefb0340f134151...,0.000000,0.000000,446878456.418966,0.000000,0.000000,1,5085.653460
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,...,2,1,11fcc769708aae4772b2ad896281286c742b88d127938f...,0.000000,0.000000,229125022.327324,0.000000,0.000000,2,2.366190
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,...,1,1,71229863ae8694da16da002e889a1d19bcc82c581225b6...,0.000000,0.000000,196018403.962452,0.000000,0.000000,1,9467.341690
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,...,3,1,115cd650b00784222bfa462e08aabef8bd082606856939...,0.000000,0.000000,199077142.860000,0.000000,0.000000,3,3.521270


In [18]:
min_holders_txs_and_assets_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bech32_address        25793 non-null  object 
 1   tx_count              25793 non-null  int64  
 2   received_sum_ada      25793 non-null  float64
 3   sent_sum_ada          25793 non-null  float64
 4   received_sum_gens     25793 non-null  float64
 5   sent_sum_gens         25793 non-null  float64
 6   received_sum_milk     25793 non-null  float64
 7   sent_sum_milk         25793 non-null  float64
 8   received_sum_min      25793 non-null  float64
 9   sent_sum_min          25793 non-null  float64
 10  received_sum_wrt      25793 non-null  float64
 11  sent_sum_wrt          25793 non-null  float64
 12  received_sum_sundae   25793 non-null  float64
 13  sent_sum_sundae       25793 non-null  float64
 14  received_token_count  25793 non-null  int64  
 15  sent_token_count   

In [19]:
min_holders_txs_and_assets_summary_df.head()

,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,received_token_count,sent_token_count,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num,ada_balance
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619873,24556163659.531025,...,9,1,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...,0.000000,0.000000,998422624.088847,0.000000,0.000000,9,5.766780
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,...,1,1,716563d77a4da81a91831c68d192a3eeefb0340f134151...,0.000000,0.000000,446878456.418966,0.000000,0.000000,1,5085.653460
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,...,2,1,11fcc769708aae4772b2ad896281286c742b88d127938f...,0.000000,0.000000,229125022.327324,0.000000,0.000000,2,2.366190
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,...,1,1,71229863ae8694da16da002e889a1d19bcc82c581225b6...,0.000000,0.000000,196018403.962452,0.000000,0.000000,1,9467.341690
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,...,3,1,115cd650b00784222bfa462e08aabef8bd082606856939...,0.000000,0.000000,199077142.860000,0.000000,0.000000,3,3.521270


# Save Final DF：

In [20]:
min_holders_txs_and_assets_summary_df.to_csv("min_holders_txs_and_assets_summary_df.csv", index=False)

# Get stake address:

In [21]:
path = "/home/jovyan/work/New Topic/min_holders_txs_and_assets_summary_df.csv"
min_holders_txs_and_assets_summary_df = pd.read_csv(path)
min_holders_txs_and_assets_summary_df.head()

,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,received_token_count,sent_token_count,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num,ada_balance
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619877,24556163659.531025,...,9,1,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...,0.000000,0.000000,998422624.088847,0.000000,0.000000,9,5.766780
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,...,1,1,716563d77a4da81a91831c68d192a3eeefb0340f134151...,0.000000,0.000000,446878456.418966,0.000000,0.000000,1,5085.653460
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,...,2,1,11fcc769708aae4772b2ad896281286c742b88d127938f...,0.000000,0.000000,229125022.327324,0.000000,0.000000,2,2.366190
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,...,1,1,71229863ae8694da16da002e889a1d19bcc82c581225b6...,0.000000,0.000000,196018403.962452,0.000000,0.000000,1,9467.341690
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,...,3,1,115cd650b00784222bfa462e08aabef8bd082606856939...,0.000000,0.000000,199077142.860000,0.000000,0.000000,3,3.521270


In [22]:
min_holders_txs_and_assets_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bech32_address        25793 non-null  object 
 1   tx_count              25793 non-null  int64  
 2   received_sum_ada      25793 non-null  float64
 3   sent_sum_ada          25793 non-null  float64
 4   received_sum_gens     25793 non-null  float64
 5   sent_sum_gens         25793 non-null  float64
 6   received_sum_milk     25793 non-null  float64
 7   sent_sum_milk         25793 non-null  float64
 8   received_sum_min      25793 non-null  float64
 9   sent_sum_min          25793 non-null  float64
 10  received_sum_wrt      25793 non-null  float64
 11  sent_sum_wrt          25793 non-null  float64
 12  received_sum_sundae   25793 non-null  float64
 13  sent_sum_sundae       25793 non-null  float64
 14  received_token_count  25793 non-null  int64  
 15  sent_token_count   

In [31]:
import requests
import json
import time
from tqdm.notebook import tqdm  

# === API config ===
API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
HEADERS = {"project_id": API_KEY}
BASE_URL = "https://cardano-mainnet.blockfrost.io/api/v0/addresses"

# === Load addresses ===
bech32_addresses = min_holders_txs_and_assets_summary_df["bech32_address"].dropna().unique().tolist()

# === Container for results ===
addr_to_stake = {}

# === Loop and call API with tqdm progress bar ===
for addr in tqdm(bech32_addresses, desc="🔍 Fetching stake addresses"):
    url = f"{BASE_URL}/{addr}/extended"
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            stake_address = data.get("stake_address")
            addr_to_stake[addr] = stake_address
        else:
            addr_to_stake[addr] = None
    except Exception as e:
        addr_to_stake[addr] = None
    time.sleep(0.3)

# === Save to JSON ===
with open("min_holders_stake_address.json", "w") as f:
    json.dump(addr_to_stake, f, indent=2)

print(f"✅ Done! {len(addr_to_stake)} addresses processed.")


🔍 Fetching stake addresses:   0%|          | 0/25793 [00:00<?, ?it/s]

✅ Done! 25793 addresses processed.


In [23]:
with open("min_holders_stake_address.json", "r") as f:
    addr_to_stake = json.load(f)

# === Convert to DataFrame ===
min_stake_df = pd.DataFrame(list(addr_to_stake.items()), columns=["bech32_address", "stake_address"])

# === Show sample
min_stake_df.head()


,bech32_address,stake_address
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,stake1u9f9v0z5zzlldgx58n8tklphu8mf7h4jvp2j2gdd...
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,None
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,stake1u9f9v0z5zzlldgx58n8tklphu8mf7h4jvp2j2gdd...
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,None
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,stake1u9f9v0z5zzlldgx58n8tklphu8mf7h4jvp2j2gdd...


In [24]:
# Count how min_stake_df stake_address are None (missing)
num_none = min_stake_df["stake_address"].isna().sum()
print(f"Number of addresses without stake_address: {num_none}")


Number of addresses without stake_address: 100


In [25]:


# === Step 1: Merge stake address into the main DataFrame ===
# min_stake_df contains columns: ["bech32_address", "stake_address"]
min_with_stake_df = min_holders_txs_and_assets_summary_df.merge(
    min_stake_df, on="bech32_address", how="left"
)

# === Step 2: Create a unified holder ID ===
# If stake_address exists → use it; otherwise use "holder::<bech32_address>"
min_with_stake_df["final_holder_id"] = min_with_stake_df.apply(
    lambda row: row["stake_address"] if pd.notna(row["stake_address"]) else f"holder::{row['bech32_address']}",
    axis=1
)

# === Step 3: Select numeric columns for aggregation ===
# Exclude address-related columns
exclude_cols = ["bech32_address", "hex_address", "stake_address", "final_holder_id"]
value_cols = [col for col in min_with_stake_df.columns if col not in exclude_cols]

# === Step 4: Group by final_holder_id and aggregate numeric values ===
aggregated_holder_df = min_with_stake_df.groupby("final_holder_id")[value_cols].sum().reset_index()

# === Step 5: (Optional) Add indicator if it's a virtual (non-staking) holder ===
aggregated_holder_df["is_virtual"] = aggregated_holder_df["final_holder_id"].str.startswith("holder::")



In [26]:
# === Step 6: (Optional) Show sample ===
aggregated_holder_df.head()

,final_holder_id,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,received_token_count,sent_token_count,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num,ada_balance,is_virtual
0,holder::addr1v82tw68ht75t439kd7k7s8ushvjsm3v25...,136,11324.432640,11129.000000,0.000000,0.000000,0.000000,0.000000,19.297118,0.000000,...,822,0,0.000000,0.000000,19.297118,0.112711,0.000062,822,195.432640,True
1,holder::addr1v82v4k5mrcyep87dnhlus24fkaw9eu0zt...,7367,3046999.942683,3045455.047936,0.000000,0.000000,0.000000,0.000000,8741895.708746,8738219.484971,...,18,18,0.000000,0.000000,3676.223775,0.000000,0.000000,17,699.643466,True
2,holder::addr1v83p9sf9xnardvpgv053ux7hpf6gdqavn...,3,7.137344,3.376212,0.000000,0.000000,0.000000,0.000000,14.757021,0.000000,...,1,0,0.000000,0.000000,14.757021,0.000000,0.000000,1,3.761132,True
3,holder::addr1v85wh6cxs73pl9nanenqlvhzyl62cyeq6...,5,71.954457,60.282977,0.000000,0.000000,0.000000,0.000000,30.000000,0.000000,...,62,0,0.000000,0.000000,30.000000,0.000000,9.114020,62,11.671480,True
4,holder::addr1v87m5rn6p4hqcm0622tfccyyexg09hxj8...,946,31488.251885,31289.453271,0.000000,0.000000,0.000000,0.000000,2378862.897125,2368070.873948,...,543,543,0.000000,0.000000,10792.023177,0.000000,0.000000,45,198.798614,True


In [27]:
min_aggregated_holder_df = aggregated_holder_df

# Save File

In [28]:
min_aggregated_holder_df.to_csv("min_aggregated_holder_df.csv", index=False)

# Get Script Tag

In [29]:
path = "/home/jovyan/work/New Topic/min_holders_txs_and_assets_summary_df.csv"
min_holders_txs_and_assets_summary_df = pd.read_csv(path)
min_holders_txs_and_assets_summary_df.head()

,bech32_address,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,received_token_count,sent_token_count,hex_address,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num,ada_balance
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,19,30.894400,25.127620,0.000000,0.000000,0.000000,0.000000,25554586283.619877,24556163659.531025,...,9,1,119ee377fa68bc3908d656dc9648b4d54612f3b200e184...,0.000000,0.000000,998422624.088847,0.000000,0.000000,9,5.766780
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,4742,6960.861190,1939.793080,0.000000,0.000000,0.000000,0.000000,656440372.292222,214567850.354104,...,1,1,716563d77a4da81a91831c68d192a3eeefb0340f134151...,0.000000,0.000000,446878456.418966,0.000000,0.000000,1,5085.653460
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,14,15.779230,13.413040,0.000000,0.000000,0.000000,0.000000,2635017700.945916,2405892678.618592,...,2,1,11fcc769708aae4772b2ad896281286c742b88d127938f...,0.000000,0.000000,229125022.327324,0.000000,0.000000,2,2.366190
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,32245,31217.829960,21684.752150,0.000000,0.000000,0.000000,0.000000,1539522515.320413,1342797550.001050,...,1,1,71229863ae8694da16da002e889a1d19bcc82c581225b6...,0.000000,0.000000,196018403.962452,0.000000,0.000000,1,9467.341690
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,10,12.227790,8.706520,0.000000,0.000000,0.000000,0.000000,997808571.440000,798731428.580000,...,3,1,115cd650b00784222bfa462e08aabef8bd082606856939...,0.000000,0.000000,199077142.860000,0.000000,0.000000,3,3.521270


In [7]:
import requests
import json
import time
from tqdm.notebook import tqdm

# === API config ===
API_KEY = "mainnetD5tXDiNzInPOA8ABiVQjb9gBJxB2zqXu"
HEADERS = {"project_id": API_KEY}
BASE_URL = "https://cardano-mainnet.blockfrost.io/api/v0/addresses"

# === Load addresses ===
bech32_addresses = min_holders_txs_and_assets_summary_df["bech32_address"].dropna().unique().tolist()

# === Container for full results ===
address_info_dict = {}

# === Loop and call API ===
for addr in tqdm(bech32_addresses, desc="🔍 Fetching full address info"):
    url = f"{BASE_URL}/{addr}"
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            data = response.json()
            address_info_dict[addr] = data  # ⬅️ Save the full response
        else:
            address_info_dict[addr] = {"error": f"Status code {response.status_code}"}
    except Exception as e:
        address_info_dict[addr] = {"error": str(e)}
    time.sleep(0.3)  # ⏱️ Respect rate limits

# === Save full data to JSON ===
with open("min_holders_full_address_info.json", "w") as f:
    json.dump(address_info_dict, f, indent=2)

print(f"✅ Done! Saved info for {len(address_info_dict)} addresses.")


🔍 Fetching full address info:   0%|          | 0/25793 [00:00<?, ?it/s]

✅ Done! Saved info for 25793 addresses.


In [30]:
import json
import pandas as pd

# === Step 1: Load full JSON data ===
with open("min_holders_full_address_info.json", "r") as f:
    full_data = json.load(f)

# === Step 2: Extract bech32_address and script ===
records = []

for addr, info in full_data.items():
    # Some records might have error
    if isinstance(info, dict) and "script" in info:
        records.append({
            "bech32_address": addr,
            "script": info["script"]
        })

# === Step 3: Convert to DataFrame ===
min_script_df = pd.DataFrame(records)

# ✅ Optional: Save to CSV
min_script_df.to_csv("min_address_script_flag.csv", index=False)

# 🔍 Preview
min_script_df.head()


,bech32_address,script
0,addr1zx0wxal6dz7rjzxk2mwfvj9564rp9uajqrscftx44...,True
1,addr1w9jk84m6fk5p4yvrr35dry4ramhmqdq0zdq4rz4w0...,True
2,addr1z87vw6ts32hywu4j4kyk9qfgd36zhzx3y7fc786vg...,True
3,addr1wy3fscaws62d59k6qqhg3xsarx7vstzczgjmdhx2j...,True
4,addr1z9wdv59sq7zzy2l6gchq3247lz7ssfsxs45nj4njh...,True


In [31]:
min_script_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25793 entries, 0 to 25792
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   bech32_address  25793 non-null  object
 1   script          25793 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 226.8+ KB


In [32]:
min_script_df.tail()

,bech32_address,script
25788,addr1q9w5anjzpgtc4c5pq29jue8f7s40tgytn4u9jd6kz...,False
25789,addr1q8yj0s58dy74h7mju3glk0snn7hm5wa87nx993ezm...,False
25790,addr1qy2mhp7dqg44xaszw5j400se4k6d8guc9csyfj4mv...,False
25791,addr1q8ha0ksmhh75a5hcpwc3yuxj4hnjzferhzk95edxj...,False
25792,addr1qxx4khtvfdx7c47ywscprw29saeu4sxyr0ytxq58s...,False


In [33]:
min_script_df["script"].value_counts()


script
False    25656
True       137
Name: count, dtype: int64

In [34]:
# === Step 1: Merge stake address into the main DataFrame ===
min_with_stake_df = min_holders_txs_and_assets_summary_df.merge(
    min_stake_df, on="bech32_address", how="left"
)

# === Step 2: Create unified holder ID ===
min_with_stake_df["final_holder_id"] = min_with_stake_df.apply(
    lambda row: row["stake_address"] if pd.notna(row["stake_address"]) else f"holder::{row['bech32_address']}",
    axis=1
)

# ✅ Step 2.5: Save all bech32_address values per final_holder_id as list
address_lookup_df = min_with_stake_df.groupby("final_holder_id")["bech32_address"].agg(list).reset_index()
address_lookup_df.rename(columns={"bech32_address": "all_bech32_addresses"}, inplace=True)

# === Step 3: Exclude address-related columns for aggregation
exclude_cols = ["bech32_address", "hex_address", "stake_address", "final_holder_id"]
value_cols = [col for col in min_with_stake_df.columns if col not in exclude_cols]

# === Step 4: Aggregate numeric values by final_holder_id
aggregated_holder_df = min_with_stake_df.groupby("final_holder_id")[value_cols].sum().reset_index()

# === Step 5: Flag virtual holders (non-staking)
aggregated_holder_df["is_virtual"] = aggregated_holder_df["final_holder_id"].str.startswith("holder::")

# ✅ Step 6: Merge back all associated bech32_address list
aggregated_holder_df = aggregated_holder_df.merge(address_lookup_df, on="final_holder_id", how="left")


In [35]:
aggregated_holder_df.head()

,final_holder_id,tx_count,received_sum_ada,sent_sum_ada,received_sum_gens,sent_sum_gens,received_sum_milk,sent_sum_milk,received_sum_min,sent_sum_min,...,sent_token_count,gens_balance,milk_balance,min_balance,wrt_balance,sundae_balance,token_num,ada_balance,is_virtual,all_bech32_addresses
0,holder::addr1v82tw68ht75t439kd7k7s8ushvjsm3v25...,136,11324.432640,11129.000000,0.000000,0.000000,0.000000,0.000000,19.297118,0.000000,...,0,0.000000,0.000000,19.297118,0.112711,0.000062,822,195.432640,True,[addr1v82tw68ht75t439kd7k7s8ushvjsm3v25jerm5ec...
1,holder::addr1v82v4k5mrcyep87dnhlus24fkaw9eu0zt...,7367,3046999.942683,3045455.047936,0.000000,0.000000,0.000000,0.000000,8741895.708746,8738219.484971,...,18,0.000000,0.000000,3676.223775,0.000000,0.000000,17,699.643466,True,[addr1v82v4k5mrcyep87dnhlus24fkaw9eu0zt7sfa2rc...
2,holder::addr1v83p9sf9xnardvpgv053ux7hpf6gdqavn...,3,7.137344,3.376212,0.000000,0.000000,0.000000,0.000000,14.757021,0.000000,...,0,0.000000,0.000000,14.757021,0.000000,0.000000,1,3.761132,True,[addr1v83p9sf9xnardvpgv053ux7hpf6gdqavn79n66uc...
3,holder::addr1v85wh6cxs73pl9nanenqlvhzyl62cyeq6...,5,71.954457,60.282977,0.000000,0.000000,0.000000,0.000000,30.000000,0.000000,...,0,0.000000,0.000000,30.000000,0.000000,9.114020,62,11.671480,True,[addr1v85wh6cxs73pl9nanenqlvhzyl62cyeq63x30r03...
4,holder::addr1v87m5rn6p4hqcm0622tfccyyexg09hxj8...,946,31488.251885,31289.453271,0.000000,0.000000,0.000000,0.000000,2378862.897125,2368070.873948,...,543,0.000000,0.000000,10792.023177,0.000000,0.000000,45,198.798614,True,[addr1v87m5rn6p4hqcm0622tfccyyexg09hxj8yr2d6el...


In [36]:
# Step 1: Create lookup dictionary from min_script_df
addr_to_script = dict(zip(min_script_df["bech32_address"], min_script_df["script"]))

# Step 2: Define tagging function
def has_script_address(address_list):
    """
    Return True if any address in the list is marked as script=True.
    Unknown addresses default to False.
    """
    return any(addr_to_script.get(addr, False) for addr in address_list)

# Step 3: Apply to aggregated_holder_df
aggregated_holder_df["has_script_address"] = aggregated_holder_df["all_bech32_addresses"].apply(has_script_address)

# Optional: Preview the result
print(aggregated_holder_df["has_script_address"].value_counts())


has_script_address
False    25077
True       128
Name: count, dtype: int64


In [37]:
aggregated_holder_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25205 entries, 0 to 25204
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   final_holder_id       25205 non-null  object 
 1   tx_count              25205 non-null  int64  
 2   received_sum_ada      25205 non-null  float64
 3   sent_sum_ada          25205 non-null  float64
 4   received_sum_gens     25205 non-null  float64
 5   sent_sum_gens         25205 non-null  float64
 6   received_sum_milk     25205 non-null  float64
 7   sent_sum_milk         25205 non-null  float64
 8   received_sum_min      25205 non-null  float64
 9   sent_sum_min          25205 non-null  float64
 10  received_sum_wrt      25205 non-null  float64
 11  sent_sum_wrt          25205 non-null  float64
 12  received_sum_sundae   25205 non-null  float64
 13  sent_sum_sundae       25205 non-null  float64
 14  received_token_count  25205 non-null  int64  
 15  sent_token_count   

# Save File CSV

In [38]:
aggregated_holder_df.to_csv("min_aggregated_holder_df.csv", index=False)